# 二、使用编码工具

## 1.　编码工具简介

HuggingFace提供了一套统一的编码API，由每个模型各自提交实现。由于统一了API，所以调用者能快速地使用不同模型的编码工具。

在学习HuggingFace的编码工具之前，先看一个示例的编码过程，以理解编码工具的工作过程。

## 2. 编码工具工作流示意

### 1) 定义字典

文字是一个抽象的概念，不是计算机擅长处理的数据单元，计算机擅长处理的是数字运算，所以需要把抽象的文字转换为数字，让计算机能够做数学运算。

为了把抽象的文字数字化，需要一个字典把文字或者词对应到某个数字。一个示意的字典如下：

In [1]:
# 字典
vocab = {
    '<SOS>': 0,
    '<EOS>': 1,
    'the': 2,
    'quick': 3,
    'brown': 4,
    'fox': 5,
    'jumps': 6,
    'over': 7,
    'a': 8,
    'lazy': 9,
    'dog': 10
}

## 2) 句子预处理

在句子被分词之前，一般会对句子进行一些特殊的操作，例如把太长的句子截短，或在句子中添加首尾标识符等。

在示例字典中，我们注意到除了一般的词之外，还有一些特殊符号，例如<SOS>和<EOS>，它们分别代表一个句子的开头和结束。把这两个特殊符号添加到句子上，代码如下：

In [2]:
# 简单编码
sent = 'the quick brown fox jumps over a lazy dog'
sent = '<SOS> ' + sent + ' <EOS>'
print(sent)

<SOS> the quick brown fox jumps over a lazy dog <EOS>


## 3) 分词

现在句子准备好了，接下来需要把句子分成一个一个的词。对于中文来讲，这是个复杂的问题，但是对于英文来讲这个问题比较容易解决，因为英文有自然的分词方式，即以空格来分词，代码如下：

In [3]:
# 英文分词
words = sent.split()
print(words)

['<SOS>', 'the', 'quick', 'brown', 'fox', 'jumps', 'over', 'a', 'lazy', 'dog', '<EOS>']


对于中文来讲，分词的问题比较复杂，因为中文所有的字是连在一起写的，不存在一个自然的分隔符号。有很多成熟的工具能够做中文分词，例如jieba分词、LTP分词等，但是在本书中不会使用这些工具，因为HuggingFace的编码工具已经包括了分词这一步工作，由各个模型自行实现，对于调用者来讲这些工作是透明的，不需要关心具体的实现细节。

### 4) 编码

句子已按要求添加了首尾标识符，并且分割成了一个一个的单词，现在需要把这些抽象的单词映射为数字。因为已经定义好了字典，所以使用字典就可以把每个单词分别地映射为数字，代码如下：

In [4]:
# 编码为数字
encode = [vocab[i] for i in words]
print(encode)

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1]


## 3. 下载Tansformers编码工具

因为编码工具包括配置文件、词典文件、预训练的模型文件和分词文件等，这些数据通常比较大，不会包含在transfomers工具包中，因此我们需要提前下载它。

这里我们用到`google-bert/bert-base-chinese`，所以我们使用huggingface-cli工具，从hf-mirror镜像下载它。

`bert-base-chinese`的主页为： https://huggingface.co/google-bert/bert-base-chinese

或国内镜像： https://hf-mirror.com/google-bert/bert-base-chinese

In [5]:
#!HF_ENDPOINT=https://hf-mirror.com hf download google-bert/bert-base-chinese --local-dir ../models/google-bert/bert-base-chinese

### 查看模型中的文件

In [6]:
! ls ../models/google-bert/bert-base-chinese

README.md	    model.safetensors  tokenizer.json
config.json	    pytorch_model.bin  tokenizer_config.json
flax_model.msgpack  tf_model.h5        vocab.txt


其中主要文件：

1) 配置文件：config.json。
2) 词典文件：vocab.txt。
3) 预训练模型文件：model.safetensors、pytorch_model.bin和tf_model.h5。
4) 分词文件：tokenizer.json和tokenizer_config.json

### 加载编码工具

In [7]:
# 加载编码工具
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('../models/google-bert/bert-base-chinese')
print(tokenizer)

/home/fspeed/HuggingFace/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertTokenizer(name_or_path='../models/google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


## 4. 准备实验数据

这是一些中文的句子，以测试编码工具

In [8]:
# 准备实验数据
sents = [
    '你站在桥上看风景',
    '看风景的人在楼上看你',
    '明月装饰了你的窗子',
    '你装饰了别人的梦'
]

## 5. 基本的编码函数

使用基本的编码函数编码

这里调用了编码工具的encode()函数，这是最基本的编码函数，一次编码一个或者一对句子，在这个例子中，编码了一对句子。

不是每个编码工具都有编码一对句子的功能，具体取决于不同模型的实现。在BERT中一般会编码一对句子，这和BERT的训练方式有关系。

1) 参数text和text_pair分别为两个句子，如果只想编码一个句子，则可让text_pair传None。
2) 参数truncation=True表明当句子长度大于max_length时，截断句子。
3) 参数padding= 'max_length'表明当句子长度不足max_length时，在句子的后面补充PAD，直到max_length长度。
4) 参数add_special_tokens=True表明需要在句子中添加特殊符号。
5) 参数max_length=25定义了max_length的长度。
6) 参数return_tensors=None表明返回的数据类型为list格式，也可以赋值为tf、pt、np，分别表示TensorFlow、PyTorch、NumPy数据格式。

In [9]:
# 基本的编码函数
out = tokenizer.encode(
      text=sents[0], #第一个句子
      text_pair=sents[1], #第二个句子
      truncation=True, #当句子长度大于max_length时截断
      padding='max_length', #一律补PAD,直到max_length长度
      add_special_tokens=True, #是否增加特殊的tokens
      max_length=25, #句子最大长度
      return_tensors=None)#返回数据的类型，可取值tf、pt、np,默认为返回list
print(out)
print(tokenizer.decode(out))

[101, 872, 4991, 1762, 3441, 677, 4692, 7599, 3250, 102, 4692, 7599, 3250, 4638, 782, 1762, 3517, 677, 4692, 872, 102, 0, 0, 0, 0]
[CLS] 你 站 在 桥 上 看 风 景 [SEP] 看 风 景 的 人 在 楼 上 看 你 [SEP] [PAD] [PAD] [PAD] [PAD]


可以看到编码的输出为一个数字的list，这里使用了编码工具的decode()函数把这个list还原为分词前的句子。这样就可以看出编码工具对句子做了哪些预处理工作。

从输出可以看出，编码工具把两个句子前后拼接在一起，中间使用[SEP]符号分隔，在整个句子的头部添加符号[CLS]，在整个句子的尾部添加符号[SEP]，因为句子的长度不足max_length，所以补充了4个[PAD]。

另外从空格的情况也能看出，编码工具把每个字作为一个词。因为每个字之间都有空格，表明它们是不同的词，所以在BERT的实现中，中文分词处理比较简单，就是把每个字都作为一个词来处理。

## 6. 进阶的编码函数

encode_plus()函数，这是一个进阶版的编码函数，它会返回更加复杂的编码结果。和encode()函数一样，encode_plus()函数也可以编码一个句子或者一对句子，在这个例子中，编码了一对句子。

参数return_token_type_ids、return_attention_mask、return_special_tokens_mask、return_length表明需要返回相应的编码结果，如果指定为False，则不会返回对应的内容。

In [10]:
# 进阶的编码函数
out = tokenizer.encode_plus(
      text=sents[0],
      text_pair=sents[1],
      truncation=True, 
      padding='max_length', 
      add_special_tokens=True, 
      max_length=25, 
      return_tensors=None, 
      return_token_type_ids=True, #返回token_type_ids
      return_special_tokens_mask=True, #返回special_tokens_mask 特殊符号标识
      return_attention_mask=True, #返回attention_mask
      return_length=True) #返回length 标识长度

#input_ids 编码后的词
#token_type_ids 第1个句子和特殊符号的位置是0,第2个句子的位置是1
#special_tokens_mask 特殊符号的位置是1,其他位置是0
#attention_mask PAD的位置是0,其他位置是1
#length 返回句子长度

print('input_ids: ', out['input_ids'])
print('将文本解码:', tokenizer.decode(out['input_ids']))

print('token_type_ids: ', out['token_type_ids'])
print('special_tokens_mask: ', out['special_tokens_mask'])
print('attention_mask: ', out['attention_mask'])
print('length: ', out['length'])

#for k, v in out.items():
#    print(k, ':', v)


input_ids:  [101, 872, 4991, 1762, 3441, 677, 4692, 7599, 3250, 102, 4692, 7599, 3250, 4638, 782, 1762, 3517, 677, 4692, 872, 102, 0, 0, 0, 0]
将文本解码: [CLS] 你 站 在 桥 上 看 风 景 [SEP] 看 风 景 的 人 在 楼 上 看 你 [SEP] [PAD] [PAD] [PAD] [PAD]
token_type_ids:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
special_tokens_mask:  [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
attention_mask:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
length:  25


看第二行，把编码结果中的input_ids还原为文字形式，可以看到经过预处理的原文本。预处理的内容和encode()函数一致。

这次编码的结果和encode()函数不一样的地方在于，这次返回的不是一个简单的list，而是4个list和1个数字，接下来对编码的结果分别进行说明。

1) 输出input_ids：编码后的词，也就是encode()函数的输出。
2) 输出token_type_ids：因为编码的是两个句子，这个list用于表明编码结果中哪些位置是第1个句子，哪些位置是第2个句子。具体表现为，第2个句子的位置是1，其他位置是0。
3) 输出special_tokens_mask：用于表明编码结果中哪些位置是特殊符号，具体表现为，特殊符号的位置是1，其他位置是0。
4) 输出attention_mask：用于表明编码结果中哪些位置是PAD。具体表现为，PAD的位置是0，其他位置是1。
5) 输出length：表明编码后句子的长度。

## 7. 批量编码函数

以上介绍的函数，都是一次编码一对或者一个句子，在实际工程中需要处理的数据往往是成千上万的，为了提高效率，可以使用batch_encode_plus ()函数批量地进行数据处理。

代码如下：

In [11]:
# 批量编码成对的句子
out = tokenizer.batch_encode_plus(
      batch_text_or_text_pairs=[(sents[0], sents[1]), (sents[2], sents[3])],
      add_special_tokens=True,
      truncation=True, #当句子长度大于max_length时截断
      padding='max_length', #一律补零,直到max_length长度
      max_length=25,
      return_tensors=None, #可取值tf、pt、np,默认为返回list
      return_token_type_ids=True, #返回token_type_ids
      return_attention_mask=True, #返回attention_mask
      return_special_tokens_mask=True, #返回special_tokens_mask 特殊符号标识
      #return_offsets_mapping=True, #返回offsets_mapping 标识每个词的起止位置,这个参数只能BertTokenizerFast使用
      return_length=True) #返回length 标识长度

#input_ids 编码后的词
#token_type_ids 第1个句子和特殊符号的位置是0,第2个句子的位置是1
#special_tokens_mask 特殊符号的位置是1,其他位置是0
#attention_mask PAD的位置是0,其他位置是1
#length 返回句子长度
for k, v in out.items():
    print(k, ':', v)
tokenizer.decode(out['input_ids'][0])

input_ids : [[101, 872, 4991, 1762, 3441, 677, 4692, 7599, 3250, 102, 4692, 7599, 3250, 4638, 782, 1762, 3517, 677, 4692, 872, 102, 0, 0, 0, 0], [101, 3209, 3299, 6163, 7652, 749, 872, 4638, 4970, 2094, 102, 872, 6163, 7652, 749, 1166, 782, 4638, 3457, 102, 0, 0, 0, 0, 0]]
token_type_ids : [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]
special_tokens_mask : [[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]
length : [21, 20]
attention_mask : [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]


'[CLS] 你 站 在 桥 上 看 风 景 [SEP] 看 风 景 的 人 在 楼 上 看 你 [SEP] [PAD] [PAD] [PAD] [PAD]'

可以看到，这里的输出都是二维的list了，表明这是一个批量的编码。

## 8. 使用自定义字典编码

### 1) 首先直接调用encode看一下分词的结果

可以看到“明月”被分成了两个词

In [12]:
# 编码新添加的词
out = tokenizer.encode(
      text='明月装饰了你的窗子[EOS]',
      text_pair=None,
      truncation=True, #当句子长度大于max_length时截断
      padding='max_length', #一律补PAD,直到max_length长度
      add_special_tokens=True,
      max_length=10,
      return_tensors=None)
print(out)
tokenizer.decode(out)

[101, 3209, 3299, 6163, 7652, 749, 872, 4638, 4970, 102]


'[CLS] 明 月 装 饰 了 你 的 窗 [SEP]'

### 2) 获取字典，然后输出结果

可以看到，字典本身是个dict类型的数据。在BERT的字典中，共有21 128个词，并且“明月”这个词并不存在于字典中。

In [13]:
# 获取字典
vocab = tokenizer.get_vocab()
print("vocab的类型：", type(vocab))
print("词表大小：", len(vocab))
print("'明月'是否在词表内：", '明月' in vocab)
print("'明'是否在词表内：", '明' in vocab)
print("'明'的序号：", vocab['明'])
print("'月'的序号：", vocab['月'])

vocab的类型： <class 'dict'>
词表大小： 21128
'明月'是否在词表内： False
'明'是否在词表内： True
'明'的序号： 3209
'月'的序号： 3299


### 3) 添加新词

既然“明月”并不存在于字典中，可以把这个新词添加到字典中

这里添加了3个新词，分别为“明月”“装饰”和“窗子”。也可以添加新的符号

In [14]:
# 添加新词
tokenizer.add_tokens(new_tokens=['明月', '装饰', '窗子'])

3

In [15]:
# 获取字典
vocab = tokenizer.get_vocab()
print("词表大小：", len(vocab))
print("'明月'是否在词表内：", '明月' in vocab)
print("'明月'的序号：", vocab['明月'])
print("'装饰'的序号：", vocab['装饰'])
print("'窗子'的序号：", vocab['窗子'])

词表大小： 21131
'明月'是否在词表内： True
'明月'的序号： 21128
'装饰'的序号： 21129
'窗子'的序号： 21130


In [16]:
# 添加新符号
tokenizer.add_special_tokens({'eos_token': '[EOS]'})

1

### 4) 再次编码查看分词结果

可以看到，“明月”已经被识别为一个词，而不是两个词，新的特殊符号[EOS]也被正确识别。

In [17]:
# 编码新添加的词
out = tokenizer.encode(
      text='明月装饰了你的窗子[EOS]',
      text_pair=None,
      truncation=True, #当句子长度大于max_length时截断
      padding='max_length', #一律补PAD,直到max_length长度
      add_special_tokens=True,
      max_length=10,
      return_tensors=None)
print(out)
tokenizer.decode(out)

[101, 21128, 21129, 749, 872, 4638, 21130, 21131, 102, 0]


'[CLS] 明月 装饰 了 你 的 窗子 [EOS] [SEP] [PAD]'

## 9. 关于Tokenizer的思考
先用tokenizer分词，看看分词后都得到了什么

In [18]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
print(input_ids)
for id in input_ids[0]:
    print(tokenizer.decode(id))

tensor([[  101,   100,  9064,  8307,  9392,  8798, 10800,  8169, 10112,  8291,
          8228,   100,  8330,  8174,   162,  8332, 10006,  8177,  9794,  8221,
         10551, 12772,  8187,   119,   100,  9510,  8233, 11643,  9586, 10600,
          8168,   119,   102]])
[CLS]
[UNK]
an
email
ap
##ol
##og
##i
##zi
##ng
to
[UNK]
for
the
t
##ra
##gi
##c
garden
##ing
mi
##sha
##p
.
[UNK]
how
it
ha
##pp
##ene
##d
.
[SEP]


#### 为什么会有那么多的#号？

In [ ]:
print(tokenizer.decode(9392))
print(tokenizer.decode(8798))
print(tokenizer.decode(10800))
print(tokenizer.decode(8169))
print(tokenizer.decode(10112))
print(tokenizer.decode(8291))

print(tokenizer.decode([9392,  8798, 10800,  8169, 10112,  8291]))

#### 让我们将几个不同的语言模型的分词结果用彩色显示出来看看

这里出了用到上面的模型，`google-bert/bert-base-chinese`还用到了
1. `google-bert/bert-base-uncased`
2. `distilbert/distilbert-base-cased-distilled-squad`
3. `distilbert/distilbert-base-uncased-finetuned-sst-2-english`
4. `openai-community/gpt2`
5. `Qwen/Qwen2-Audio-7B`

用以下命令一起都下载了：

In [ ]:
#!HF_ENDPOINT=https://hf-mirror.com hf download google-bert/bert-base-uncased --local-dir ../models/google-bert/bert-base-uncased
#!HF_ENDPOINT=https://hf-mirror.com hf download distilbert/distilbert-base-cased-distilled-squad --local-dir ../models/distilbert/distilbert-base-cased-distilled-squad
#!HF_ENDPOINT=https://hf-mirror.com hf download distilbert/distilbert-base-uncased-finetuned-sst-2-english --local-dir ../models/distilbert/distilbert-base-uncased-finetuned-sst-2-english
#!HF_ENDPOINT=https://hf-mirror.com hf download openai-community/gpt2 --local-dir ../models/openai-community/gpt2
#!HF_ENDPOINT=https://hf-mirror.com hf download Qwen/Qwen2-Audio-7B --local-dir ../models/Qwen/Qwen2-Audio-7B

In [ ]:
from transformers import AutoTokenizer
colors_list = [
'102;194;165', '252;141;98', '141;160;203',
'231;138;195', '166;216;84', '255;217;47'
]
def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    print(tokenizer.decode(token_ids))
    for idx, t in enumerate(token_ids):
        print(f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' + tokenizer.decode(t) + '\x1b[0m', end=' ')

In [ ]:
sentence = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened. 明月装饰了你的窗子"
show_tokens(sentence, '../models/google-bert/bert-base-chinese')

In [ ]:
sentence = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened. 明月装饰了你的窗子"
show_tokens(sentence, '../models/google-bert/bert-base-uncased')

In [ ]:
sentence = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened. 明月装饰了你的窗子"
show_tokens(sentence, '../models/distilbert/distilbert-base-cased-distilled-squad')

In [ ]:
sentence = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened. 明月装饰了你的窗子"
show_tokens(sentence, '../models/distilbert/distilbert-base-uncased-finetuned-sst-2-english')

In [ ]:
sentence = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened. 明月装饰了你的窗子"
show_tokens(sentence, '../models/openai-community/gpt2')

In [ ]:
sentence = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened. 明月装饰了你的窗子"
show_tokens(sentence, '../models/Qwen/Qwen2-Audio-7B')

#### 上面的乱码是怎么回事？

In [ ]:
# 这样没有乱码
sentence = "明月装饰了你的窗子"
show_tokens(sentence, '../models/Qwen/Qwen2-Audio-7B')

In [ ]:
# 这样就有乱码了
sentence = " 明月装饰了你的窗子"
show_tokens(sentence, '../models/Qwen/Qwen2-Audio-7B')

In [ ]:
# 是不是空格造成的呢？
sentence = " 明"
show_tokens(sentence, '../models/Qwen/Qwen2-Audio-7B')

In [ ]:
# 我们发现“ 明”被分成了两个token
sentence = " 明"
tokenizer = AutoTokenizer.from_pretrained('../models/Qwen/Qwen2-Audio-7B')
token_ids = tokenizer(sentence).input_ids
print(token_ids)
#ord(char)

In [ ]:
# 这两个token分别decode后得到三个字符（这三个字符毫无意义）
print(ord(tokenizer.decode([38903])[0]))
print(ord(tokenizer.decode([38903])[1]))
print(ord(tokenizer.decode([236])[0]))

In [ ]:
# 但是如果我们把这两个token一起decode，则生成了两个字符。我们去网上查一下uncode编码“明”字对应的就是26126
# https://www.lddgo.net/string/cjk-unicode
print(ord(tokenizer.decode([38903, 236])[0]))
print(ord(tokenizer.decode([38903, 236])[1]))
